In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

### HWP 문서 가져오기



https://nomadamas.gitbook.io/ragchain-docs/ragchain-structure/file-loader/win32-hwp-loader


In [2]:
# from RAGchain.preprocess.loader import Win32HwpLoader
from RAGchain.preprocess.loader import RustHwpLoader

loader = RustHwpLoader("./data/2024년도 하반기 중남미 지역기구 파견인턴 선발 공고.hwp")

# loader = Win32HwpLoader("./data/(첨부5) 2024년도 하반기 중남미 지역기구 파견인턴 선발 공고.hwpx")
documents = loader.load()

len(documents)

10

In [3]:
from pprint import pprint
pprint(documents[0].page_content)

(' 2024년도 하반기 중남미 지역기구 파견 인턴 선발 공고   1. 선발 개요     가. 공통 자격요건     ㅇ 대한민국 국적자로, '
 '국내외 대학생(3-4학년) 또는 대학원생    ㅇ 영어 또는 스페인어로 업무처리가 가능한 자      (학력) ※ 위 학력사항은 기본 '
 '요건이며, 기구별 상세요건 확인 요망 ※ 기구별 학력요건은 본 공고 4페이지 “참고1” 참조  (어학) ※ 어학능력 증빙 자료(어학성적, '
 '자격증 등) 제출 요망  ※ 증빙자료가 없을 시 언어 능력 뒷받침할 수 있는 내용을 신청서에 기재 ※ 기구별 어학요건은 본 공고 4페이지 '
 '“참고1” 참조  (기타 고려사항) ※ 대한민국 정부가 지원하는 유사 인턴 프로그램 기참가자는 지원 불가 ※ 비수도권대학 학생의 지원 '
 '장려(총 인원의 30%이상 비수도권대학에서 선발할 계획. 다만, 비수도권대학 학생 지원율 저조 등 사유가 있는 경우 동 목표 비율은 '
 '탄력적으로 조정될 예정)    *비수도권대학 판단 기준은 국가법령정보센터(www.law.go.kr)에 게시된 ‘균형인사지침’ 참고 ※ '
 '백신 접종 완료 여부 확인을 위한 백신접종완료증명서 사본 제출      나. 기구별 선발 기간 및 인원     ㅇ 파견기간은 원칙적으로 '
 '4~6개월이나, 기구 사정에 따라 변동 가능        - OAS(미주기구)의 경우 선발 후 25년 상반기 파견    ㅇ 기구별 선호 '
 '상세 지원자격은 본 공고 4페이지 “참고1” 참조  기구명 파견지 파견 기간 인원 PARLATINO(라틴아메리카의회) 파나마 6개월 1 '
 'IOM(국제이주기구) 페루 6개월 1 OPANAL(중남미핵무기금지기구) 멕시코 2024.7월 ~ 11월 1 GGGI(글로벌녹색성장연구소) '
 '파라과이 추후 조율 1 SEGIB(이베로아메리카사무국) 스페인 2024.9월 ~ 2025.2월 1 CAF(중남미개발은행) 콜롬비아 '
 '2024.9월 ~ 2025.3월 1 ACTO(아마존협력조약기구) 브라질 6개월

In [4]:
pprint(documents[1].page_content)

(',(학력),※ 위 학력사항은 기본 요건이며, 기구별 상세요건 확인 요망,※ 기구별 학력요건은 본 공고 4페이지 “참고1” '
 '참조,,(어학),※ 어학능력 증빙 자료(어학성적, 자격증 등) 제출 요망 ,※ 증빙자료가 없을 시 언어 능력 뒷받침할 수 있는 내용을 '
 '신청서에 기재,※ 기구별 어학요건은 본 공고 4페이지 “참고1” 참조,,(기타 고려사항),※ 대한민국 정부가 지원하는 유사 인턴 프로그램 '
 '기참가자는 지원 불가,※ 비수도권대학 학생의 지원 장려(총 인원의 30%이상 비수도권대학에서 선발할 계획. 다만, 비수도권대학 학생 '
 '지원율 저조 등 사유가 있는 경우 동 목표 비율은 탄력적으로 조정될 예정),   *비수도권대학 판단 기준은 '
 '국가법령정보센터(www.law.go.kr)에 게시된 ‘균형인사지침’ 참고,※ 백신 접종 완료 여부 확인을 위한 백신접종완료증명서 사본 '
 '제출,')


## Simple RAG
### Indexing

In [5]:
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate

In [6]:
# HugoingFace Embeddings를 다운로드
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model_name = "BM-K/KoSimCSE-roberta-multitask"

embeddings_model = HuggingFaceEmbeddings(
    model_name=embedding_model_name,  
)

# HugoingFace Embedding 모델의 Tokenizer를 사용하여 토큰화
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(embedding_model_name)

/home/samuel/Dev/langchain-kr/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
No sentence-transformers model found with name BM-K/KoSimCSE-roberta-multitask. Creating a new one with mean pooling.


In [7]:

embeddings_model

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='BM-K/KoSimCSE-roberta-multitask', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [8]:
# Token 수를 기준으ㄹ 문서를 청크 단위로 분할
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer = tokenizer,
    chunk_size = 500,
    chunk_overlap  = 50,
)

split_docs = text_splitter.split_documents(documents)
print(len(split_docs))
print(split_docs[0])

27
page_content='2024년도 하반기 중남미 지역기구 파견 인턴 선발 공고   1. 선발 개요     가. 공통 자격요건     ㅇ 대한민국 국적자로, 국내외 대학생(3-4학년) 또는 대학원생    ㅇ 영어 또는 스페인어로 업무처리가 가능한 자      (학력) ※ 위 학력사항은 기본 요건이며, 기구별 상세요건 확인 요망 ※ 기구별 학력요건은 본 공고 4페이지 “참고1” 참조  (어학) ※ 어학능력 증빙 자료(어학성적, 자격증 등) 제출 요망  ※ 증빙자료가 없을 시 언어 능력 뒷받침할 수 있는 내용을 신청서에 기재 ※ 기구별 어학요건은 본 공고' metadata={'source': './data/2024년도 하반기 중남미 지역기구 파견인턴 선발 공고.hwp', 'page_type': 'text'}


In [9]:
vectorstore = Chroma.from_documents(documents=split_docs, 
                                    embedding=embeddings_model)

### Retreival

In [10]:
# VectorStore를 사용하여 검색기 생성
retriever = vectorstore.as_retriever(search_kwargs={'k': 2})

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# Chat Model
llm = ChatOllama(model="gemma:7b")


def format_docs(docs):
    # 중복을 제거
    unique_contents = set(doc.page_content for doc in docs)
    
    # \n\n으로 연결하여 문자열로 반환
    return "\n\n".join(unique_contents)


# RAG Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Chain 실행
response = rag_chain.invoke("파견 기간은 얼마인가요?")
print(response)

파견 기간은 원칙적으로 4~6개월이나, 기구 사정에 따라 변동 가능합니다.


In [11]:
retriever.get_relevant_documents("파견 기간은 얼마인가요?")

/home/samuel/Dev/langchain-kr/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(metadata={'page_type': 'text', 'source': './data/2024년도 하반기 중남미 지역기구 파견인턴 선발 공고.hwp'}, page_content='나. 기구별 선발 기간 및 인원     ㅇ 파견기간은 원칙적으로 4~6개월이나, 기구 사정에 따라 변동 가능        - OAS(미주기구)의 경우 선발 후 25년 상반기 파견    ㅇ 기구별 선호 상세 지원자격은 본 공고 4페이지 “참고1” 참조  기구명 파견지 파견 기간 인원 PARLATINO(라틴아메리카의회) 파나마 6개월 1 IOM(국제이주기구) 페루 6개월 1 OPANAL(중남미핵무기금지기구) 멕시코 2024.7월 ~ 11월 1 GGGI(글로벌녹색성장연구소) 파라과이 추후 조율 1 SEGIB(이베로아메리카사무국) 스페인 2024.9월 ~ 2025.2월 1 CAF(중남미개발은행) 콜롬비아 2024.9월'),
 Document(metadata={'page_type': 'table', 'source': './data/2024년도 하반기 중남미 지역기구 파견인턴 선발 공고.hwp'}, page_content='기구명,파견지,파견 기간,인원,PARLATINO(라틴아메리카의회),파나마,6개월,1,IOM(국제이주기구),페루,6개월,1,OPANAL(중남미핵무기금지기구),멕시코,2024.7월 ~ 11월,1,GGGI(글로벌녹색성장연구소),파라과이,추후 조율,1,SEGIB(이베로아메리카사무국),스페인,2024.9월 ~ 2025.2월,1,CAF(중남미개발은행),콜롬비아,2024.9월 ~ 2025.3월,1,ACTO(아마존협력조약기구),브라질,6개월,1,OAS(미주기구),미국(워싱턴),2025.1월 ~ 2025.7월,1,ACS(카리브국가연합),트리니다드토바고,6개월,1')]

In [12]:
response = rag_chain.invoke("필기시험이 있는 날짜는 언제인가요?")
print(response)

필기시험의 날짜는 2024년 6월 4일(화)입니다.


In [13]:
retriever.get_relevant_documents("필기시험이 있는 날짜는 언제인가요?")

[Document(metadata={'page_type': 'text', 'source': './data/2024년도 하반기 중남미 지역기구 파견인턴 선발 공고.hwp'}, page_content='참고   다. 지원 내역 : 왕복 항공료(정액), 현지 정착비 및 월 체재비      ※ 다만, 지원금으로 충당할 수 없는 파견 기구 및 파견국의 물가 등에 따라 발생하는 기타 비용은 지원자 부담 필요   2. 선발 일정     ㅇ 2024.5.30.(목) 18:00서류접수 마감(이메일 접수)     ㅇ 2024.5.31.(금)  서류전형 합격자 발표(개별 통보)    ㅇ 2024.6.4.(화)       필기시험 및 면접시험(상세내용 추후 공지)    ㅇ 2024.6.10.(월) 외교부 추천 후보자 발표(개별 통보)'),
 Document(metadata={'page_type': 'table', 'source': './data/2024년도 하반기 중남미 지역기구 파견인턴 선발 공고.hwp'}, page_content='참고2  ,, 필기시험 평가항목 및 배점')]

In [14]:
response = rag_chain.invoke("국제이주기구 파견은 언제 하나요?")
print(response)

국제이주기구 파견은 6개월 동안입니다.


In [15]:
retriever.get_relevant_documents("국제이주기구 파견은 언제 하나요?")


[Document(metadata={'page_type': 'text', 'source': './data/2024년도 하반기 중남미 지역기구 파견인턴 선발 공고.hwp'}, page_content='나. 기구별 선발 기간 및 인원     ㅇ 파견기간은 원칙적으로 4~6개월이나, 기구 사정에 따라 변동 가능        - OAS(미주기구)의 경우 선발 후 25년 상반기 파견    ㅇ 기구별 선호 상세 지원자격은 본 공고 4페이지 “참고1” 참조  기구명 파견지 파견 기간 인원 PARLATINO(라틴아메리카의회) 파나마 6개월 1 IOM(국제이주기구) 페루 6개월 1 OPANAL(중남미핵무기금지기구) 멕시코 2024.7월 ~ 11월 1 GGGI(글로벌녹색성장연구소) 파라과이 추후 조율 1 SEGIB(이베로아메리카사무국) 스페인 2024.9월 ~ 2025.2월 1 CAF(중남미개발은행) 콜롬비아 2024.9월'),
 Document(metadata={'page_type': 'table', 'source': './data/2024년도 하반기 중남미 지역기구 파견인턴 선발 공고.hwp'}, page_content='기구명,파견지,파견 기간,인원,PARLATINO(라틴아메리카의회),파나마,6개월,1,IOM(국제이주기구),페루,6개월,1,OPANAL(중남미핵무기금지기구),멕시코,2024.7월 ~ 11월,1,GGGI(글로벌녹색성장연구소),파라과이,추후 조율,1,SEGIB(이베로아메리카사무국),스페인,2024.9월 ~ 2025.2월,1,CAF(중남미개발은행),콜롬비아,2024.9월 ~ 2025.3월,1,ACTO(아마존협력조약기구),브라질,6개월,1,OAS(미주기구),미국(워싱턴),2025.1월 ~ 2025.7월,1,ACS(카리브국가연합),트리니다드토바고,6개월,1')]

In [16]:
retriever.get_relevant_documents("IOM")

[Document(metadata={'page_type': 'text', 'source': './data/2024년도 하반기 중남미 지역기구 파견인턴 선발 공고.hwp'}, page_content='나. 기구별 선발 기간 및 인원     ㅇ 파견기간은 원칙적으로 4~6개월이나, 기구 사정에 따라 변동 가능        - OAS(미주기구)의 경우 선발 후 25년 상반기 파견    ㅇ 기구별 선호 상세 지원자격은 본 공고 4페이지 “참고1” 참조  기구명 파견지 파견 기간 인원 PARLATINO(라틴아메리카의회) 파나마 6개월 1 IOM(국제이주기구) 페루 6개월 1 OPANAL(중남미핵무기금지기구) 멕시코 2024.7월 ~ 11월 1 GGGI(글로벌녹색성장연구소) 파라과이 추후 조율 1 SEGIB(이베로아메리카사무국) 스페인 2024.9월 ~ 2025.2월 1 CAF(중남미개발은행) 콜롬비아 2024.9월'),
 Document(metadata={'page_type': 'table', 'source': './data/2024년도 하반기 중남미 지역기구 파견인턴 선발 공고.hwp'}, page_content='기구명,파견지,파견 기간,인원,PARLATINO(라틴아메리카의회),파나마,6개월,1,IOM(국제이주기구),페루,6개월,1,OPANAL(중남미핵무기금지기구),멕시코,2024.7월 ~ 11월,1,GGGI(글로벌녹색성장연구소),파라과이,추후 조율,1,SEGIB(이베로아메리카사무국),스페인,2024.9월 ~ 2025.2월,1,CAF(중남미개발은행),콜롬비아,2024.9월 ~ 2025.3월,1,ACTO(아마존협력조약기구),브라질,6개월,1,OAS(미주기구),미국(워싱턴),2025.1월 ~ 2025.7월,1,ACS(카리브국가연합),트리니다드토바고,6개월,1')]

### Table Data

In [17]:
for d in split_docs:
    if "IOM" in d.page_content:
        print(d.page_content)
        sample_text = d.page_content
        break

나. 기구별 선발 기간 및 인원     ㅇ 파견기간은 원칙적으로 4~6개월이나, 기구 사정에 따라 변동 가능        - OAS(미주기구)의 경우 선발 후 25년 상반기 파견    ㅇ 기구별 선호 상세 지원자격은 본 공고 4페이지 “참고1” 참조  기구명 파견지 파견 기간 인원 PARLATINO(라틴아메리카의회) 파나마 6개월 1 IOM(국제이주기구) 페루 6개월 1 OPANAL(중남미핵무기금지기구) 멕시코 2024.7월 ~ 11월 1 GGGI(글로벌녹색성장연구소) 파라과이 추후 조율 1 SEGIB(이베로아메리카사무국) 스페인 2024.9월 ~ 2025.2월 1 CAF(중남미개발은행) 콜롬비아 2024.9월


#### HTML 테이블 구조를 컨텍스트로 전달

In [18]:
sample_text

'나. 기구별 선발 기간 및 인원     ㅇ 파견기간은 원칙적으로 4~6개월이나, 기구 사정에 따라 변동 가능        - OAS(미주기구)의 경우 선발 후 25년 상반기 파견    ㅇ 기구별 선호 상세 지원자격은 본 공고 4페이지 “참고1” 참조  기구명 파견지 파견 기간 인원 PARLATINO(라틴아메리카의회) 파나마 6개월 1 IOM(국제이주기구) 페루 6개월 1 OPANAL(중남미핵무기금지기구) 멕시코 2024.7월 ~ 11월 1 GGGI(글로벌녹색성장연구소) 파라과이 추후 조율 1 SEGIB(이베로아메리카사무국) 스페인 2024.9월 ~ 2025.2월 1 CAF(중남미개발은행) 콜롬비아 2024.9월'

In [19]:
len(tokenizer.encode(sample_text))

175

In [20]:
llm.invoke(f"""Answer the question based only on the following context:
{sample_text}

Question: 국제이주기구 파견은 언제 하나요?
""")

AIMessage(content='국제이주기구 파견은 2024년 7월부터 2024년 11월까지입니다.', response_metadata={'model': 'gemma:7b', 'created_at': '2024-07-10T00:52:14.331880088Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 7696163865, 'load_duration': 1133629, 'prompt_eval_count': 289, 'prompt_eval_duration': 1371645000, 'eval_count': 33, 'eval_duration': 6153516000}, id='run-84d9877a-1510-4ff9-aa7a-c85c6b689f36-0')

#### Markdown 형식으로 테이블 데이터를 변환하여 컨텍스트로 전달

In [21]:
import pandas as pd
from bs4 import BeautifulSoup

# BeautifulSoup을 사용하여 HTML 파싱
soup = BeautifulSoup(sample_text, 'html.parser')

# 테이블 찾기
table = soup.find('table')

# Pandas DataFrame으로 변환
df = pd.read_html(str(table))[0]

# DataFrame을 Markdown으로 변환
markdown_table = df.to_markdown(index=False)

print(markdown_table)

/tmp/ipykernel_19990/272752229.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


ValueError: No tables found

In [ ]:
llm.invoke(f"""Answer the question based only on the following context:
{markdown_table}

Question: 국제이주기구 파견은 언제 하나요?
""")

In [ ]:
len(tokenizer.encode(markdown_table))

#### Text 와 Table 구분하여 청크로 분할

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup

def convert_html_table_to_markdown(html_table_text: str) -> str:

    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(html_table_text, 'html.parser')

    # 테이블 찾기
    table = soup.find('table')

    # Pandas DataFrame으로 변환
    df = pd.read_html(str(table))[0]

    # DataFrame을 Markdown으로 변환
    markdown_table = df.to_markdown(index=False)

    return markdown_table

In [ ]:
documents[0].metadata

In [ ]:
markdown_documents = []

for doc in documents:
    if doc.metadata.get("page_type")=='table':

        try:
            doc.page_content = convert_html_table_to_markdown(doc.page_content)
            markdown_documents.append(doc)
        except:
            markdown_documents.append(doc)
    else:
        markdown_documents.append(doc)


len(markdown_documents)

In [ ]:
new_split_docs = text_splitter.split_documents(markdown_documents)
print(len(new_split_docs))
print(new_split_docs[0])

In [ ]:
vectorstore2 = Chroma.from_documents(documents=new_split_docs, 
                                    embedding=embeddings_model)


retriever2 = vectorstore2.as_retriever(search_kwargs={'k': 2})

rag_chain2 = ( 
    {"context": retriever2 | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = rag_chain2.invoke("국제이주기구 파견은 언제 하나요?")
response

In [ ]:
retriever2.get_relevant_documents("국제이주기구 파견은 언제 하나요?")